In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# import sys
# sys.path.insert(0, '../')
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""


import numpy as np
import tensorflow as tf

from gantools import data
from gantools import utils
from gantools import plot
from gantools.model import WGAN, LapWGAN, UpscalePatchWGAN, UpscalePatchWGANBorders
from gantools.gansystem import GANsystem
from gantools.data import fmap, transformation
from gantools import evaluation
import functools
import matplotlib.pyplot as plt
from copy import deepcopy

In [ ]:
Nsamples = 16
global_path = 'saved_results'

In [ ]:
def upscale_1d(system, signal_low, border=None):
    Nsignal, dlow = signal_low.shape
    ps = system.params['net']['shape'][0]//2
    s = system.params['net']['upsampling']
    print('The input signal is of size: {}'.format(signal_low.shape))
    signal_low = transformation.upsamler_1d(signal_low, s, Nwin=32)
    print('Upsampling to size: {}'.format(signal_low.shape))
    if border is None:
        border = np.zeros([Nsignal, ps, 1])
    
    signal_up = np.zeros([Nsignal,s*dlow])
    with tf.Session() as sess:
        system.load(sess)
        for i in range(s*dlow//ps):
            X_down = np.expand_dims(signal_low[:,i*ps:(i+1)*ps],axis=2)
            border = system.generate(X_smooth=X_down, borders=border, sess=sess)
            signal_up[:,i*ps:(i+1)*ps] = border[:,:,0]
    return signal_up

# Low dim part 2048

In [ ]:
low_time_str = 'piano_2028_phase_shuffle'
downscale_2k = 16
name = 'WGAN' + '_' + low_time_str
pathmodel2k = os.path.join(global_path, name+'_checkpoints')

In [ ]:
wgan2k = utils.load_gan(pathmodel2k, model=WGAN)
gen_sample_2k = np.squeeze(wgan2k.generate(Nsamples))

In [ ]:
dataset2k = data.load.load_audio_dataset(scaling=downscale_2k, type='piano', spix=2048, augmentation=True)
real_sample_2k = dataset2k.get_samples(N=Nsamples)

In [ ]:
for i in range(4):
    print('Generated sample {}'.format(i))
    plot.audio.play_sound(gen_sample_2k[i,:], fs=16000//downscale_2k)
    
for i in range(4):
    print('Real sample {}'.format(i))
    plot.audio.play_sound(real_sample_2k[i,:], fs=16000//downscale_2k)

# Up sampling 8k

In [ ]:
downscale_8k = 4
name = 'WGAN_piano_8k_patch2'
pathmodel8k = os.path.join(global_path, name+'_checkpoints')

In [ ]:
dataset8k = data.load.load_audio_dataset(scaling=downscale_8k, type='piano', spix=4*2048, augmentation=True)
real_sample_8k = dataset8k.get_samples(N=Nsamples)

In [ ]:
wgan8k = utils.load_gan(pathmodel8k, model=UpscalePatchWGANBorders)
gen_sample_8k = upscale_1d(wgan8k, gen_sample_2k)

In [ ]:
for i in range(4):
    print('Generated sample {}'.format(i))
    plot.audio.play_sound(gen_sample_8k[i,:], fs=16000//downscale_8k)
    
for i in range(4):
    print('Real sample {}'.format(i))
    plot.audio.play_sound(real_sample_8k[i,:], fs=16000//downscale_8k)

# Up sampling 32k

In [ ]:
downscale_32k = 1
name = 'WGAN_piano_32k_patch'
pathmodel32k = os.path.join(global_path, name+'_checkpoints')

In [ ]:
dataset32k = data.load.load_audio_dataset(scaling=downscale_32k, type='piano', spix=16*2048, augmentation=True)
real_sample_32k = dataset32k.get_samples(N=Nsamples)

In [ ]:
wgan32k = utils.load_gan(pathmodel32k, model=UpscalePatchWGANBorders)
gen_sample_32k = upscale_1d(wgan32k, gen_sample_8k)

In [ ]:
for i in range(4):
    print('Generated sample {}'.format(i))
    plot.audio.play_sound(gen_sample_32k[i,:], fs=16000//downscale_32k)
    
for i in range(4):
    print('Real sample {}'.format(i))
    plot.audio.play_sound(real_sample_32k[i,:], fs=16000//downscale_32k)

# Long signal

In [ ]:
def gen_1d(system, N, length=2048*8, border=None):
    ps = system.params['net']['shape'][0]//2
    if border is None:
        border = np.zeros([N, ps, 1])
    
    signal_up = np.zeros([N,length])
    with tf.Session() as sess:
        system.load(sess)
        for i in range(length//ps):
            border = system.generate(borders=border, sess=sess)
            signal_up[:,i*ps:(i+1)*ps] = border[:,:,0]
    return signal_up

In [ ]:
downscale_2k = 16
pathmodel = os.path.join(global_path, 'WGAN_piano_2028_rnn_checkpoints')
length = 2048*8

In [ ]:
wganlong2k = utils.load_gan(pathmodel, model=UpscalePatchWGANBorders)
gen_sample_2k_long = gen_1d(wganlong2k, N=Nsamples, length=length)

In [ ]:
dataset2k_long = data.load.load_audio_dataset(scaling=downscale_2k, type='piano', spix=length, augmentation=True)
real_sample_2k_long = dataset2k_long.get_samples(N=Nsamples)

In [ ]:
for i in range(4):
    print('Generated sample {}'.format(i))
    plot.audio.play_sound(gen_sample_2k_long[i,:], fs=16000//downscale_2k)
    
for i in range(4):
    print('Real sample {}'.format(i))
    plot.audio.play_sound(real_sample_2k_long[i,:], fs=16000//downscale_2k)

In [ ]:
wgan8k = utils.load_gan(pathmodel8k, model=UpscalePatchWGANBorders)
gen_sample_8k_long = upscale_1d(wgan8k, gen_sample_2k_long)

wgan8k = utils.load_gan(pathmodel8k, model=UpscalePatchWGANBorders)
gen_sample_8k_long_single = upscale_1d(wgan8k, real_sample_2k_long)

# wgan32k = utils.load_gan(pathmodel32k, model=UpscalePatchWGANBorders)
# gen_sample_32k_long = upscale_1d(wgan32k, gen_sample_8k_long)

In [ ]:
dataset8k_long = data.load.load_audio_dataset(scaling=downscale_8k, type='piano', spix=length*4, augmentation=True)
real_sample_8k_long = dataset8k_long.get_samples(N=Nsamples)

In [ ]:
for i in range(4):
    print('Generated sample {}'.format(i))
    plot.audio.play_sound(gen_sample_8k_long[i,:], fs=16000//downscale_8k)
    
for i in range(4):
    print('Real sample {}'.format(i))
    plot.audio.play_sound(real_sample_8k_long[i,:], fs=16000//downscale_8k)
    
for i in range(4):
    print('Single step - sample {}'.format(i))
    plot.audio.play_sound(gen_sample_8k_long_single[i,:], fs=16000//downscale_8k)